# Extract respiratory signals

This notebook extracts respiratory signals from the given scenarios using the pixel intensity and optical flow methods.

In [ ]:
import os
import numpy as np
import respiration.dataset as repository

dataset = repository.from_default()
scenarios = dataset.get_scenarios(['101_natural_lighting'])
scenarios

In [ ]:
results_dir = os.path.join(os.getcwd(), '..', 'evaluation', 'unsupervised')
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [ ]:
from datetime import datetime

parameters = {
    'quality_level': 0.1,
    'quality_level_rv': 0.05,
    'use_cgof': True,
}

evaluation_metadata = {
    'timestamp_start': datetime.now(),
    'scenarios': scenarios,
    'parameters': parameters,
}

In [ ]:
import respiration.roi as roi

yolo = roi.YOLO()


def get_rois(frame: np.ndarray) -> list[tuple[np.ndarray, str]]:
    """
    Get the regions of interest (ROIs) for the given frame
    :param frame: The frame to get the ROIs from
    :return: A list of tuples containing the ROI and the name of the ROI
    """

    regions = [
        # ROI for the full frame
        ((0, 0, frame.shape[1], frame.shape[0]), 'full')
    ]

    # Calculate the region of interest (ROI) based on the face
    faces = roi.detect_faces(frame)
    if len(faces) == 1:
        chest_roi = roi.roi_from_face(faces[0])
        regions.append((chest_roi, 'chest'))

    # Use the detected person to create a mask
    persons = yolo.detect_classes(frame, clazz='person')
    if len(persons) == 1:
        regions.append((persons[0], 'person'))

    return regions

In [ ]:
from respiration.extractor import pixel_intensity, optical_flow

extracted_signals = []

for (subject, setting) in scenarios:
    print(f'Processing {subject} - {setting}')

    frames, params = dataset.get_video_gray(subject, setting, False)

    rois = get_rois(frames[0])

    for region in rois:
        roi_area, roi_name = region

        #
        # Calculate the average pixel intensity
        #

        pi_start = datetime.now()
        pi_signal = pixel_intensity.average_pixel_intensity(frames, roi=roi_area)
        extracted_signals.append({
            'subject': subject,
            'setting': setting,
            'method': 'pixel_intensity',
            'roi': roi_name,
            'roi_area': list(roi_area),
            'execution_time': datetime.now() - pi_start,
            'sample_rate': params.fps,
            'signal': pi_signal.tolist(),
        })

        #
        # Calculate the optical flow
        #

        of_cgof_start = datetime.now()
        of_signal_raw = optical_flow.extract_signal(
            frames,
            roi=roi_area,
            quality_level=parameters['quality_level'],
            quality_level_rv=parameters['quality_level_rv'],
            use_cgof=parameters['use_cgof'],
        )

        extracted_signals.append({
            'subject': subject,
            'scenario': frames,
            'method': 'optical_flow',
            'roi': roi_name,
            'roi_area': list(roi_area),
            'execution_time': datetime.now() - of_cgof_start,
            'sample_rate': params.fps,
            'signal': of_signal_raw.tolist(),
        })

    # Garbage collect the frames
    del frames

In [ ]:
evaluation_metadata['timestamp_finish'] = datetime.now()

In [ ]:
import pandas as pd

df = pd.DataFrame(extracted_signals)
df.head()

In [ ]:
import respiration.utils as utils

# Save the extracted_signals as a JSON
json_path = os.path.join(results_dir, 'predictions.json')
utils.write_json(json_path, extracted_signals)

# Save the evaluation dataframe
csv_path = os.path.join(results_dir, 'predictions.csv')
df.to_csv(csv_path, index=False)

# Save the hyperparameters as prettified json
json_path = os.path.join(results_dir, 'parameters.json')
utils.write_json(json_path, evaluation_metadata)